<a href="https://colab.research.google.com/github/superaja/covid19/blob/master/covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import requests
import io
import plotly.graph_objects as go

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-19-2020.csv')
df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
#df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')
#df_time_series = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

In [0]:
# Data Prep
df_by_country = df.groupby('Country/Region').sum() # remove state/province
df_by_country['Country/Region'] = df_by_country.index
# filter out countries with less than 100 confirmed cases
df_country = df_by_country[df_by_country['Confirmed'] > 100]

In [0]:
df_by_country['Not Yet Recovered'] = df_by_country['Confirmed'] - df_by_country['Deaths'] - df_by_country['Recovered']
df_by_country['% Recovered'] = round(df_by_country['Recovered'] / df_by_country['Confirmed'], 2)*100
df_by_country['% Mortality'] = round(df_by_country['Deaths'] / df_by_country['Confirmed'], 2)*100
df_by_country['% Not Yet Recovered'] = round(df_by_country['Not Yet Recovered'] / df_by_country['Confirmed'], 2)*100

In [0]:
country = list(df_by_country['Country/Region'].unique())

In [0]:
# Core Metrics by Country
def covid_core_metrics():
  country = ['Italy', 'US', 'Spain', 'China', 'Korea, South', 'United Kingdom']
  #country = list(df_by_country['Country/Region'].unique())
  x = df_by_country[df_by_country['Country/Region'].isin(country)]['Country/Region']
  y = df_by_country[df_by_country['Country/Region'].isin(country)]['% Recovered']
  y1 = df_by_country[df_by_country['Country/Region'].isin(country)]['% Mortality']
  y2 = df_by_country[df_by_country['Country/Region'].isin(country)]['% Not Yet Recovered']
  fig = go.Figure(data=go.Bar(x=x, y=y, name='% Recovered', text=y, textposition='auto', marker_color='green'))
  fig.add_trace(go.Bar(x=x, y=y1, name='% Mortality', text=y1, textposition='auto'))
  fig.add_trace(go.Bar(x=x, y=y2, name='% Not Yet Recovered', text=y2, textposition='auto', marker_color='rgb(144, 202, 249)'))
  #fig.add_trace(go.Scatter(x=x_days, y=y_us, mode='lines+markers+text', name='US Recovered', text=y_us, textposition='top left'))
  fig.show()



In [0]:
# Confirmed Case Tracker
df_clean = df_time_series.drop(columns=['Province/State', 'Lat', 'Long'])
country = ['Italy', 'US', 'Spain', 'China', 'Korea, South', 'United Kingdom']
def days_since_100_confirmed():
  fig = go.Figure()
  for i in country:
    df_country = df_clean[df_clean['Country/Region'] == i]
    df_country = df_country.groupby('Country/Region').sum()
    country_confirmed = df_country.iloc[0].tolist()
    country_confirmed_gt_100 = [i for i in country_confirmed[1:] if i>100]
    label = [round(i/1000, 2) for i in country_confirmed_gt_100]
    days_since_100 = [i for i in range(1, len(country_confirmed_gt_100))]
    fig.add_trace(go.Scatter(x=days_since_100, y=country_confirmed_gt_100, mode='lines+markers', name=i+ ' Confirmed', \
                             text=label, textposition='top left'))
  fig.update_layout(title='Confirmed Cases by Country',
                    xaxis_title='Number of Days since 100 Confirmed Cases',
                    yaxis_title= 'Confirmed Cases',
                    yaxis_type='log')
  fig.show()

In [17]:
covid_core_metrics()
days_since_100_confirmed()

In [0]:
country = ['US', 'Italy', 'Spain']
x = df_by_country[df_by_country['Country/Region'].isin(country)]['Country/Region']
y = df_by_country[df_by_country['Country/Region'].isin(country)]['% Recovered']

In [0]:
df_time_series.drop(columns=['Lat', 'Long'], inplace=True)

In [0]:
countries = list(df['Country/Region'].unique())
us_states = list(df[df['Country/Region']=='US']['Province/State'].unique())

In [0]:
df_us_confirmed = df_time_series[df_time_series['Country/Region']=='US']
df_italy_confirmed = df_time_series[df_time_series['Country/Region']=='Italy']
df_china_confirmed = df_time_series[df_time_series['Country/Region']=='China']

In [0]:
df_us_confirmed = df_us_confirmed.groupby('Country/Region').sum()
df_italy_confirmed = df_italy_confirmed.groupby('Country/Region').sum()
df_china_confirmed = df_china_confirmed.groupby('Country/Region').sum()

In [0]:
x_days = list(df_china_confirmed.columns)

In [11]:
df_italy_confirmed

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Italy,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,7,10,12,17,21,29,34,52,79,107,148,197,233,366,463,631,827,827,1266,1441,1809,2158,2503,2978


In [0]:
y = df_italy_confirmed.values.tolist()
y_us = df_us_confirmed.values.tolist()
y_china = df_china_confirmed.values.tolist()


In [0]:
y =y[0]
y_us = y_us[0]
y_china = y_china[0]
#del y[0:4]
#del y_us[0:4]
#del y_china[0:4]

In [0]:
def rate_calc(y):
  rate = [0]
  i=0
  #while i < len(y)-1:
  for i in range(len(y)-1):
    if y[i] == 0:
      temp_rate = 0
    else: 
      temp_rate = round((y[i+1] - y[i]) / y[i], 1)*100
    rate.append(temp_rate)
    i = i+1

  rate_label = [str(i) + '%' for i in rate]
  return rate, rate_label
#format 

#y = [round(i/1000, 0) for i in y]
#y_us = [round(i/1000, 0) for i in y_us]
#y_china = [round(i/1000, 0) for i in y_china]

In [0]:
rate, rate_label = rate_calc(y)

In [16]:
fig = go.Figure(data=go.Scatter(x=x_days, y=y, mode='lines+markers+text', name='Italy Recovered', text=y, textposition='top left'))
fig.add_trace(go.Scatter(x=x_days, y=y_us, mode='lines+markers+text', name='US Recovered', text=y_us, textposition='top left'))
#fig.add_trace(go.Scatter(x=x_days, y=y_china, mode='lines+markers+text', name='China Recovered', text=y_china, textposition='top left'))
fig.show()

In [17]:
fig1 = go.Figure(data=go.Scatter(x=x_days, y=rate, mode='lines+markers+text', name='Italy Mortality Rates', text=rate_label, textposition='top left'))
fig1.show()

In [0]:
df_us = df[df['Country/Region']=='US']
df_us_ts = df_time_series[df_time_series['Country/Region']=='US']

In [19]:
df_us_ts.drop(columns=['Country/Region'], inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
some = df_us_ts.T
header = some.iloc[0]
some = some[1:]
some.rename(columns = header)

,Washington,New York,California,Massachusetts,Diamond Princess,Grand Princess,Georgia,Colorado,Florida,New Jersey,Oregon,Texas,Illinois,Pennsylvania,Iowa,Maryland,North Carolina,South Carolina,Tennessee,Virginia,Arizona,Indiana,Kentucky,District of Columbia,Nevada,New Hampshire,Minnesota,Nebraska,Ohio,Rhode Island,Wisconsin,Connecticut,Hawaii,Oklahoma,Utah,Kansas,Louisiana,Missouri,Vermont,Alaska,...,"Dallas, TX","Tarrant, TX","Montgomery, TX","Middlesex, NJ","Jefferson, CO","Multnomah, OR","Polk, OR","Deschutes, OR","McHenry, IL","Lake, IL","Bucks, PA","Hanover, VA","Lancaster, SC","Sullivan, TN","Johnson, IN","Howard, IN","St. Joseph, IN","Knox, NE","Stark, OH","Anoka, MN","Olmsted, MN","Summit, UT","Fairfield, CT","Litchfield, CT","Orleans, LA","Pennington, SD","Beadle, SD","Charles Mix, SD","Davison, SD","Minnehaha, SD","Bon Homme, SD","Socorro, NM","Bernalillo, NM","Oakland, MI","Wayne, MI","New Castle, DE",Alabama,Puerto Rico,Guam,Virgin Islands
1/22/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/23/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/24/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/25/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/26/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/27/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/28/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/29/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/30/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1/31/20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
